In [13]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
from tqdm import notebook

In [2]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
data = pd.concat([positive, negative]).reset_index(drop=True)
data.tail(10)

,text,label
226824,"@ivanenko14 и у меня также, только будильник е...",negative
226825,Когда же я вернусь к тебе(( http://t.co/36qnCO...,negative
226826,"RT @qelasocadij: Скажите пожалуйста, как у чел...",negative
226827,У нас физ ра на улице\nПака линт:(\nЧерез 45 м...,negative
226828,"Нас сегодня отказались принять в сад, типа мы ...",negative
226829,Но не каждый хочет что то исправлять:( http://...,negative
226830,скучаю так :-( только @taaannyaaa вправляет мо...,negative
226831,"Вот и в школу, в говно это идти уже надо(",negative
226832,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
226833,Такси везет меня на работу. Раздумываю приплат...,negative


In [3]:
def remove_user(pattern, repl, string):
    return re.sub(pattern, repl, string)
data.text = np.vectorize(remove_user)('@[\w]*', '', data.text)
data.text.head()

0     хоть я и школота, но поверь, у нас то же само...
1    Да, все-таки он немного похож на него. Но мой ...
2          RT : Ну ты идиотка) я испугалась за тебя!!!
3    RT : "Кто то в углу сидит и погибает от голода...
4     Вот что значит страшилка :D\nНо блин,посмотре...
Name: text, dtype: object

In [4]:
data['text'] = data['text'].apply(str)


In [5]:
data.to_csv("text_in.txt", header=None, index=None, sep=',', mode='w', encoding='utf-8')

In [6]:
text_in = open('text_in.txt','r', encoding='utf-8')

0.8323495
0.5288512
0.21199903


In [7]:
import gensim.downloader as api

In [8]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [9]:
word_vectors = api.load("glove-wiki-gigaword-100")  # download pre-trained words vectors from gensim-data
# lets get the nearest to  'woman', 'king' and the farest from 'man'
result = word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.7699


In [17]:
# determine the words similarity
similarity = word_vectors.similarity('woman', 'man')
print(similarity)

similarity = word_vectors.similarity('human', 'man')
print(similarity)

similarity = word_vectors.similarity('bee', 'man')
print(similarity)

0.8323495
0.5288512
0.21199903


In [11]:
# get the top-3 the closest words
result = word_vectors.similar_by_word("man", topn=3)
print(result)

result = word_vectors.similar_by_word("cat", topn=3)
print(result)

result = word_vectors.similar_by_word("mouth", topn=3)
print(result)

[('woman', 0.8323495388031006), ('boy', 0.7914870977401733), ('one', 0.7788748741149902)]
[('dog', 0.8798074722290039), ('rabbit', 0.7424427270889282), ('cats', 0.732300341129303)]
[('tongue', 0.7366125583648682), ('mouths', 0.6877489686012268), ('ear', 0.6811771392822266)]


In [14]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("text_in.txt", "r", encoding='utf-8') as fin:
        for line in notebook.tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [15]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [16]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("text_in.txt", "r",  encoding='utf-8') as fin:
    for line in notebook.tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

0it [00:00, ?it/s]

In [18]:
sentences = [i for i in sentences if len(i) > 2]

In [19]:
sentences[0]

['школотый',
 'поверь',
 'самый',
 'd',
 'общество',
 'профилировать',
 'предмет',
 'типаpositive']

In [21]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=0)

In [22]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in notebook.tqdm(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        
        if counter > 100000:
            break

w2v_index.build(10)
ft_index.build(10)

0it [00:00, ?it/s]

True

In [23]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, ) #return 5 nearest vectors
    return [index_map[i] for i in answers]

In [24]:
TEXT = "хоть я и школота, но поверь, у нас то же самое общество профилирующий предмет типа" 

In [25]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D",positive\n',
 '"Надеялась, что я добавлю ее в друзья,лохушка :D http://t.co/36k6ADyASA",positive\n',
 '"с утра романтик, засевший глубоко среди извилин мозга, получил суровым кирпичом действительности по голове) и вас с добры",positive\n',
 '"Кто круче - 1,2,3,4 ?) #Suri http://t.co/jZ0INfN4tx",positive\n',
 '"И вот я сижу .. А он передо мной ..в трусах , а я засыпаю :D",positive\n']

In [26]:
get_response(TEXT, ft_index, modelFT, index_map)

['"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D",positive\n',
 '"с утра романтик, засевший глубоко среди извилин мозга, получил суровым кирпичом действительности по голове) и вас с добры",positive\n',
 '"RT : По телеку показывали ""Квартал-95"". Они пригласили Чайф! =)) Наконец-то догадались, что Киркоров и Моисеев не нужны. Вообщ",positive\n',
 '"Кто круче - 1,2,3,4 ?) #Suri http://t.co/jZ0INfN4tx",positive\n',
 '"И вот я сижу .. А он передо мной ..в трусах , а я засыпаю :D",positive\n']

In [30]:
TEXT = "Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D"

In [32]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['"с утра романтик, засевший глубоко среди извилин мозга, получил суровым кирпичом действительности по голове) и вас с добры",positive\n',
 '"Кто круче - 1,2,3,4 ?) #Suri http://t.co/jZ0INfN4tx",positive\n',
 '"И вот я сижу .. А он передо мной ..в трусах , а я засыпаю :D",positive\n',
 '"время двенадцати нету, а я уже дома:)\n',
 '"Надеялась, что я добавлю ее в друзья,лохушка :D http://t.co/36k6ADyASA",positive\n']

In [31]:
get_response(TEXT, ft_index, modelFT, index_map)

['"Надеялась, что я добавлю ее в друзья,лохушка :D http://t.co/36k6ADyASA",positive\n',
 ':(((",negative\n',
 '"с утра романтик, засевший глубоко среди извилин мозга, получил суровым кирпичом действительности по голове) и вас с добры",positive\n',
 'НАДОЕЛО!!!",negative\n',
 '"RT : По телеку показывали ""Квартал-95"". Они пригласили Чайф! =)) Наконец-то догадались, что Киркоров и Моисеев не нужны. Вообщ",positive\n']